In [1]:
from IPython.core.display import HTML
import requests
import time
import pandas as pd
import datetime
import pytz

In [2]:
import os
import numpy as np
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/master project/data/NVDA' #เข้า drive

Mounted at /content/drive


In [3]:
api_key = 'J4AM5YKF88UA41WO'

In [4]:
def _get_data(symbols, time_from, time_to, api_key):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbols}&time_from={time_from}&time_to={time_to}&limit=1000&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    return data

In [5]:
def get_dataset(time_from="20190101T0000",
                time_to="20250301T0000",
                MAX_API_CALLS_PER_DAY = 99999, # Free tier only allows 25 API calls per day
                MAX_API_CALLS_PER_MIN = 300 # Free tier only allows 5 api calls per minute
               ):
    data_list=[]
    for i in range(1,MAX_API_CALLS_PER_DAY+1):
        if i%600==0:
            time.sleep(65)

        data=_get_data('NVDA',time_from,time_to,api_key)
        if 'feed' not in data:break
        if len(data['feed'])==0: break
        data_list.append(data)
        time_to=data['feed'][-1]['time_published'][:-2] # Take all the way up to last 2 since api only takes minute level granularity
    df=pd.concat([pd.DataFrame(data['feed']) for data in data_list])
    # Extract NVDA specific relevance (we didn't use it in video)

    # Only take tickers with NVDA in headline
    df=df[df.title.str.contains('NVDA|NVIDIA',case=False)]
    # Only take when # of tickers = 1
    # Drop duplicates..
    df.drop_duplicates(subset=['summary'],inplace=True,keep='first')
    # Set index to time published
    #df.set_index('time_published',inplace=True)
    # Sort by time published
    #df.sort_index(inplace=True)
    return df


In [6]:
df = get_dataset()

In [7]:
df['time_published'] = pd.to_datetime(df['time_published']).dt.tz_localize('UTC')

In [8]:
df['time_published']=df['time_published'].dt.tz_convert('US/Eastern')

In [9]:
df['time_published'] = pd.to_datetime(df['time_published']).dt.strftime('%Y-%m-%d')

In [10]:
df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,"Tariffs Shock Hit Stocks, Cryptos, While Nvidi...",https://www.benzinga.com/markets/cryptocurrenc...,2025-02-28,[Piero Cingari],The high-flying artificial intelligence sector...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.084409,Neutral,"[{'ticker': 'AMD', 'relevance_score': '0.07469..."
4,Nvidia-Backed CoreWeave Eyes $4 Billion IPO: I...,https://www.benzinga.com/25/02/44053318/nvidia...,2025-02-28,[Surbhi Jain],"Nvidia-backed CoreWeave targets $4B IPO, eyein...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'IPO', 'relevance_score': '0.576289...",0.102035,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.2121..."
11,"Nvidia, Tesla Lead Tech Stock Rebound: What's ...",https://www.benzinga.com/general/market-summar...,2025-02-28,[Erica Kollmann],"Nvidia and Tesla rebound on Friday, leading te...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,General,www.benzinga.com,"[{'topic': 'Life Sciences', 'relevance_score':...",0.072192,Neutral,"[{'ticker': 'NVDA', 'relevance_score': '0.2373..."
15,Bitcoin's Sharp Decline Pulls Crypto Market Do...,https://www.benzinga.com/25/02/44044656/bitcoi...,2025-02-28,[Akanksha Bakshi],"Bitcoin Will Still Hit $500,000 Before Trump L...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.027200,Neutral,"[{'ticker': 'BABA', 'relevance_score': '0.0408..."
16,Investigating NVIDIA's Standing In Semiconduct...,https://www.benzinga.com/insights/news/25/02/4...,2025-02-28,[Benzinga Insights],In the fast-paced and cutthroat world of busin...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.230226,Somewhat-Bullish,"[{'ticker': 'NVDA', 'relevance_score': '0.5344..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,"Semiconductor Watchlist: Nvidia, Broadcom, Pan...",https://www.thestreet.com/investing/semiconduc...,2022-03-07,[Brian O'Connell],Chip stocks see gyrations tied to Ukraine inva...,https://www.thestreet.com/.image/ar_16:9%2Cc_f...,The Street,n/a,www.thestreet.com,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.087795,Neutral,"[{'ticker': 'AMD', 'relevance_score': '0.06851..."
237,Why Nvidia Stock Tanked Today,https://www.fool.com/investing/2022/03/07/why-...,2022-03-07,[Rich Smith],Goldman Sachs kind of likes Nvidia -- but can'...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.303647,Somewhat-Bearish,"[{'ticker': 'GS', 'relevance_score': '0.366061..."
239,"Why AMD, NVIDIA And Taiwan Semiconductor Manuf...",https://www.benzinga.com/markets/cryptocurrenc...,2022-03-07,[Henry Khederian],Shares of several companies in the broader tec...,,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.015156,Neutral,"[{'ticker': 'AMD', 'relevance_score': '0.76151..."
248,Here's How Goldman Sachs Views NVIDIA,https://www.benzinga.com/analyst-ratings/analy...,2022-03-07,[],Goldman Sachs analyst Toshiya Hari reinstated ...,https://cdn.benzinga.com/files/imagecache/og_i...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.014886,Neutral,"[{'ticker': 'GS', 'relevance_score': '0.323078..."


In [11]:
News_NVDA = df[["time_published", "title", "summary","overall_sentiment_score"]]

In [12]:
News_NVDA

,time_published,title,summary,overall_sentiment_score
0,2025-02-28,"Tariffs Shock Hit Stocks, Cryptos, While Nvidi...",The high-flying artificial intelligence sector...,-0.084409
4,2025-02-28,Nvidia-Backed CoreWeave Eyes $4 Billion IPO: I...,"Nvidia-backed CoreWeave targets $4B IPO, eyein...",0.102035
11,2025-02-28,"Nvidia, Tesla Lead Tech Stock Rebound: What's ...","Nvidia and Tesla rebound on Friday, leading te...",0.072192
15,2025-02-28,Bitcoin's Sharp Decline Pulls Crypto Market Do...,"Bitcoin Will Still Hit $500,000 Before Trump L...",0.027200
16,2025-02-28,Investigating NVIDIA's Standing In Semiconduct...,In the fast-paced and cutthroat world of busin...,0.230226
...,...,...,...,...
235,2022-03-07,"Semiconductor Watchlist: Nvidia, Broadcom, Pan...",Chip stocks see gyrations tied to Ukraine inva...,-0.087795
237,2022-03-07,Why Nvidia Stock Tanked Today,Goldman Sachs kind of likes Nvidia -- but can'...,-0.303647
239,2022-03-07,"Why AMD, NVIDIA And Taiwan Semiconductor Manuf...",Shares of several companies in the broader tec...,-0.015156
248,2022-03-07,Here's How Goldman Sachs Views NVIDIA,Goldman Sachs analyst Toshiya Hari reinstated ...,-0.014886


In [13]:
!pip install nltk

In [14]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [15]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [16]:
sia = SentimentIntensityAnalyzer()
# Analyze sentiment for each summary

x = str(News_NVDA['summary']).encode("utf-8")
News_NVDA['sentiment_score'] = News_NVDA['summary'].apply(lambda x: sia.polarity_scores(x)['compound'])

<ipython-input-16-9173405d2b7f>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  News_NVDA['sentiment_score'] = News_NVDA['summary'].apply(lambda x: sia.polarity_scores(x)['compound'])


In [17]:
News_NVDA

,time_published,title,summary,overall_sentiment_score,sentiment_score
0,2025-02-28,"Tariffs Shock Hit Stocks, Cryptos, While Nvidi...",The high-flying artificial intelligence sector...,-0.084409,0.5574
4,2025-02-28,Nvidia-Backed CoreWeave Eyes $4 Billion IPO: I...,"Nvidia-backed CoreWeave targets $4B IPO, eyein...",0.102035,0.5106
11,2025-02-28,"Nvidia, Tesla Lead Tech Stock Rebound: What's ...","Nvidia and Tesla rebound on Friday, leading te...",0.072192,0.7845
15,2025-02-28,Bitcoin's Sharp Decline Pulls Crypto Market Do...,"Bitcoin Will Still Hit $500,000 Before Trump L...",0.027200,-0.4939
16,2025-02-28,Investigating NVIDIA's Standing In Semiconduct...,In the fast-paced and cutthroat world of busin...,0.230226,0.2500
...,...,...,...,...,...
235,2022-03-07,"Semiconductor Watchlist: Nvidia, Broadcom, Pan...",Chip stocks see gyrations tied to Ukraine inva...,-0.087795,-0.4404
237,2022-03-07,Why Nvidia Stock Tanked Today,Goldman Sachs kind of likes Nvidia -- but can'...,-0.303647,-0.2291
239,2022-03-07,"Why AMD, NVIDIA And Taiwan Semiconductor Manuf...",Shares of several companies in the broader tec...,-0.015156,0.4939
248,2022-03-07,Here's How Goldman Sachs Views NVIDIA,Goldman Sachs analyst Toshiya Hari reinstated ...,-0.014886,0.0000


In [18]:
News_NVDA_Nozero=News_NVDA.loc[News_NVDA['sentiment_score'] != 0]
News_NVDA_Nozero

,time_published,title,summary,overall_sentiment_score,sentiment_score
0,2025-02-28,"Tariffs Shock Hit Stocks, Cryptos, While Nvidi...",The high-flying artificial intelligence sector...,-0.084409,0.5574
4,2025-02-28,Nvidia-Backed CoreWeave Eyes $4 Billion IPO: I...,"Nvidia-backed CoreWeave targets $4B IPO, eyein...",0.102035,0.5106
11,2025-02-28,"Nvidia, Tesla Lead Tech Stock Rebound: What's ...","Nvidia and Tesla rebound on Friday, leading te...",0.072192,0.7845
15,2025-02-28,Bitcoin's Sharp Decline Pulls Crypto Market Do...,"Bitcoin Will Still Hit $500,000 Before Trump L...",0.027200,-0.4939
16,2025-02-28,Investigating NVIDIA's Standing In Semiconduct...,In the fast-paced and cutthroat world of busin...,0.230226,0.2500
...,...,...,...,...,...
234,2022-03-07,"Seeing More Spam, Phishing or Malware? Ask Nvi...",You may be at the highest risk if you are runn...,-0.292198,-0.2732
235,2022-03-07,"Semiconductor Watchlist: Nvidia, Broadcom, Pan...",Chip stocks see gyrations tied to Ukraine inva...,-0.087795,-0.4404
237,2022-03-07,Why Nvidia Stock Tanked Today,Goldman Sachs kind of likes Nvidia -- but can'...,-0.303647,-0.2291
239,2022-03-07,"Why AMD, NVIDIA And Taiwan Semiconductor Manuf...",Shares of several companies in the broader tec...,-0.015156,0.4939


In [19]:
daily_avg_sentiment_NVDA = News_NVDA.groupby('time_published')['sentiment_score'].mean().reset_index()
daily_avg_sentiment_NVDA

,time_published,sentiment_score
0,2022-03-04,-0.273200
1,2022-03-07,-0.089760
2,2022-03-08,0.132933
3,2022-03-09,0.079600
4,2022-03-10,-0.063767
...,...,...
1023,2025-02-24,0.183818
1024,2025-02-25,0.359616
1025,2025-02-26,0.333462
1026,2025-02-27,0.477856


In [20]:
daily_avg_sentiment_Nozero_NVDA = News_NVDA_Nozero.groupby('time_published')['sentiment_score'].mean().reset_index()
daily_avg_sentiment_Nozero_NVDA

,time_published,sentiment_score
0,2022-03-04,-0.273200
1,2022-03-07,-0.112200
2,2022-03-08,0.299100
3,2022-03-09,0.106133
4,2022-03-10,-0.076520
...,...,...
995,2025-02-24,0.212842
996,2025-02-25,0.455513
997,2025-02-26,0.436065
998,2025-02-27,0.509713


In [21]:
daily_avg_sentiment_NVDA.to_csv(os.path.join(path,'daily_avg_sentiment_NVDA.csv'))
News_NVDA.to_csv(os.path.join(path,'News_NVDA.csv'))

In [22]:
daily_avg_sentiment_Nozero_NVDA.to_csv(os.path.join(path,'daily_avg_sentiment_Nozero_NVDA.csv'))
News_NVDA_Nozero.to_csv(os.path.join(path,'News_NVDA_Nozero.csv'))